In [ ]:
# Import libraries and modules
# Import libraries and modules
import numpy as np
import time

np.random.seed(123) # for reproducibility
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Conv2DTranspose, Reshape, Dense, Activation,Flatten, Dropout, Convolution2D, MaxPooling2D, Input
from utilitaire import affiche


##################################################
# I - Load pre-shuffled MNIST data train and test sets
##################################################
from tensorflow.keras.datasets.mnist import load_data
from matplotlib import pyplot


 Il y a 60 000 images de 28x28 pixels dans la base de données. Lese labels sont des entiers entre 0 et 9.
 Les images sont des matrices de pixels de taille 28x28. Chaque pixel est un entier entre 0 et 255. : ils representent un chiffre de 0 a 9.


 - TRAIN = 60 000 images
 - TEST = 10 000 images

In [ ]:

# load dataset
(X_train, y_train), (X_test, y_test) = load_data() # Chargement des données de MNIST

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)

print("===========================================")
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

Permet de ne pas prendre toute la base !!!!

In [ ]:
#on divise la base de train en 2 : 10% pour la validation et 90% pour l'entrainement
X_train, pipo, y_train, pipo = train_test_split(X_train, y_train, test_size=0.9)

#on divise la base de test en 2 : 10% pour la validation et 90% pour l'entrainement
X_test, pipo, y_test, pipo = train_test_split(X_test, y_test, test_size=0.9)

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)

print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

#pipo
print("pipo original shape", pipo.shape)


#shape d'une image
print(X_train[1,:].shape)



In [ ]:

for i in range(2):
    plt.subplot(1,2,i+1)
    plt.imshow(X_train[i,:].reshape([28,28]), cmap='gray')
    plt.axis('off')
    plt.show()


In [ ]:
#AVANT : (60000, 28, 28)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
#APRES : (60000, 28, 28, 1)

X_test = X_test.reshape(X_test.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


Y_train = tf.keras.utils.to_categorical(y_train, 10)
Y_test = tf.keras.utils.to_categorical(y_test, 10)
#ainsi, la classe 1 devient [0,1,0,0,0,0,0,0,0,0]
#la classe 2 devient [0,0,1,0,0,0,0,0,0,0] ... alors avant c'était juste 1,2,3,4,5,6,7,8,9 ...




# Question

1) Le code load la base MNSIT avec ses 60 000 images. On divise la base en 2 parties : une partie d'apprentissage (train) et une partie de test (test). On affiche la taille de la base d'apprentissage et de test.

2) Il y a 10 classes, de 0 a 9. 

3) Pré traitement :

- On reshape les images
- On les met en float
- On les normalise
- on les met en one hot encoding


Les reshape permettent de mettre les images en 28x28x1. pour avoir la profondeur (niveau de gris) de l'image.
Les images sont en float pour pouvoir les comparer entre autres reseaux de neurones. (avoir la mm précision + utilise moins de mémoire vive)


Cela permet de faciliter l'apprentissage du reseau de neurones.


4) la fonction tf.keras.utils.to_categorical permet de faire du one hot encoding. au lieu d'avoir un label de 0 a 9, on a un vecteur de 10 cases avec des 0 et un 1 a la case correspondant au label.

exemple : 5 -> [0,0,0,0,0,1,0,0,0,0]

# Creation d'un réseau de neuronnes

- ici, on cree un reseau de neurones avec 2 couches cachées de 512 neurones chacune.


- L'activation softmax permet de normaliser les valeurs de sortie entre 0 et 1. Cela permet de les interpreter comme des probabilités.
- La commande Flatten permet de mettre les images en 1D. (28x28x1 -> 784x1)

In [ ]:
inputs = Input(shape=(28,28,1))
x = inputs
print(x.shape)
x=Flatten()(x)
print(x.shape)
outputs=Dense(10, activation='softmax')(x)

#cela signifie que l'on a 28*28 neurones en entrée et 10 neurones en sortie
model = Model(inputs, outputs)
model.summary()

#nombre de param
print("Nombre de paramètres : ", model.count_params())

# nombre de param par le calcul
print(28*28*10 + 10)



In [ ]:
lr= 0.1
batch_size= 32
epochs=50

#on definit le type de descente de gradient
sgd1= tf.keras.optimizers.SGD(learning_rate=lr)


#cela permet de calculer l'erreur de classification
model.compile(loss='categorical_crossentropy', optimizer=sgd1, metrics=['accuracy'])
tps1 = time.time()


history =model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_data=(X_test, Y_test))
tps2 = time.time()
#print(history.history.keys())

affiche(history)
print('lr=',lr,'batch_size=',batch_size, 'epochs=',epochs)
print('Temps d apprentissage',tps2 - tps1)


- lr : learning rate , c'est le pas d'apprentissage. C'est la taille du pas que l'on fait pour trouver le minimum de la fonction de cout.

- batch_size : nombre d'images que l'on va donner au reseau de neurones avant de faire une mise a jour des poids.

- epochs : nombre de fois que l'on va faire passer toutes les images dans le reseau de neurones.


__Fonction de cout : categorical_crossentropy__

**categorical_crossentropy: Used as a loss function for multi-class classification model where there are two or more output labels. The output label is assigned one-hot category encoding value in form of 0s and 1. The output label, if present in integer form, is converted into categorical encoding using keras.utils to_categorical method.**

Ici, la categorical_crossentropy est utilisée car on a 10 classes. (0 a 9)



La fonction affiche(history) permet d'afficher les courbes d'apprentissage. On peut voir l'evolution de la fonction de cout et de la precision en fonction du nombre d'epochs.


# Evaluation du modèle

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))



1) model.predict permet de predire les labels des images de test. 

2) on utilise y_pred.argmax car on a fait du one hot encoding. On veut donc recuperer la position du 1 dans le vecteur.



# CNN : Sans momentum


On ajoute une couche cachée a 256 neuronnes.

In [ ]:
inputs = Input(shape=(28,28,1))
x = inputs
x=Flatten()(x)
x=Dense(256, activation='relu')(x)
outputs=Dense(10, activation='softmax')(x)

model_couche_cachee = Model(inputs, outputs)



#nombre de param
print("Nombre de paramètres : ", model_couche_cachee.count_params())
print(256*(28*28+1) + 10*(256+1))

In [ ]:
lr= 0.01
batch_size= 64
epochs=50

#on definit le type de descente de gradient : on ajoute le momentum

sgd1= tf.keras.optimizers.SGD(learning_rate=lr)


#cela permet de calculer l'erreur de classification
model_couche_cachee.compile(loss='categorical_crossentropy', optimizer=sgd1, metrics=['accuracy'])
tps1 = time.time()


history =model_couche_cachee.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_data=(X_test, Y_test))
tps2 = time.time()
#print(history.history.keys())

affiche(history)


In [ ]:
score = model_couche_cachee.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))


# Ajout momentum

In [ ]:
lr= 0.01
batch_size= 64
epochs=50

#on definit le type de descente de gradient : on ajoute le momentum

sgd1= tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)


#cela permet de calculer l'erreur de classification
model_couche_cachee.compile(loss='categorical_crossentropy', optimizer=sgd1, metrics=['accuracy'])
tps1 = time.time()


history =model_couche_cachee.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_data=(X_test, Y_test))
tps2 = time.time()
#print(history.history.keys())



In [ ]:
score = model_couche_cachee.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))


# Avec Drop OUT

In [ ]:
inputs = Input(shape=(28,28,1))
x = inputs
x=Dropout(0.4)(x)
x=Flatten()(x)
x=Dense(256, activation='relu')(x)

outputs=Dense(10, activation='softmax')(x)

model_couche_cachee_dropout = Model(inputs, outputs)



#nombre de param
print("Nombre de paramètres : ", model_couche_cachee_dropout.count_params())
print(256*(28*28+1) + 10*(256+1))


lr= 0.1
batch_size= 32
epochs=50

#on definit le type de descente de gradient : on ajoute le momentum

sgd1= tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)


#cela permet de calculer l'erreur de classification
model_couche_cachee_dropout.compile(loss='categorical_crossentropy', optimizer=sgd1, metrics=['accuracy'])
tps1 = time.time()


history =model_couche_cachee_dropout.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_data=(X_test, Y_test))
tps2 = time.time()
#print(history.history.keys())

affiche(history)
print('lr=',lr,'batch_size=',batch_size, 'epochs=',epochs)
print('Temps d apprentissage',tps2 - tps1)


print("=========================================== EVALUATION ===========================================")
score = model_couche_cachee_dropout.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

le drop out evite l'overfitting !!


# Ajout d'une seconde couche

In [ ]:
inputs = Input(shape=(28,28,1))
x = inputs
x=Flatten()(x)
x=Dense(256, activation='relu')(x)
x=Dense(256, activation='relu')(x)
x=Dropout(0.5)(x)
outputs=Dense(10, activation='softmax')(x)

model_2_couche_cachee_dropout = Model(inputs, outputs)



#nombre de param
print("Nombre de paramètres : ", model_2_couche_cachee_dropout.count_params())


lr= 0.1
batch_size= 128
epochs=50

#on definit le type de descente de gradient : on ajoute le momentum

sgd1= tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)


#cela permet de calculer l'erreur de classification
model_2_couche_cachee_dropout.compile(loss='categorical_crossentropy', optimizer=sgd1, metrics=['accuracy'])
tps1 = time.time()


history =model_2_couche_cachee_dropout.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_data=(X_test, Y_test))
tps2 = time.time()
#print(history.history.keys())

affiche(history)
print('lr=',lr,'batch_size=',batch_size, 'epochs=',epochs)
print('Temps d apprentissage',tps2 - tps1)

#evaluation


score = model_2_couche_cachee_dropout.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

# CNN

In [ ]:
inputs = Input(shape=(28,28,1))
x = inputs
#ajout convolution
x=Convolution2D(32, kernel_size=(3,3), activation='relu')(x)
x=Convolution2D(64, kernel_size=(3,3), activation='relu')(x)
x=Flatten()(x)
x=Dense(256, activation='relu')(x)
x=Dense(256, activation='relu')(x)
outputs=Dense(10, activation='softmax')(x)

model_CNN = Model(inputs, outputs)

print("Nombre de paramètres : ", model_CNN.count_params())

model_CNN.summary()

print("couche 1 a ", 32*(3*3*1+1), " paramètres")
print("couche 2 a ", 64*(3*3*32+1), " paramètres")

print("MLP a ", 256*(9216+1), " paramètres")


In [ ]:

lr= 0.01
batch_size= 64
epochs=50

#on definit le type de descente de gradient : on ajoute le momentum

sgd1= tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)


#cela permet de calculer l'erreur de classification
model_CNN.compile(loss='categorical_crossentropy', optimizer=sgd1, metrics=['accuracy'])
tps1 = time.time()


history =model_CNN.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_data=(X_test, Y_test))

affiche(history)

#evaluation


score = model_CNN.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

 # Ajout Max Pooling

In [ ]:
inputs = Input(shape=(28,28,1))
x = inputs
#ajout convolution
x=Convolution2D(32, kernel_size=(3,3), activation='relu')(x)
x=Convolution2D(64, kernel_size=(3,3), activation='relu')(x)
#max pooling
x=MaxPooling2D(pool_size=(2,2))(x)
x=Flatten()(x)
x=Dense(256, activation='relu')(x)
x=Dense(256, activation='relu')(x)
outputs=Dense(10, activation='softmax')(x)

model_CNN_max = Model(inputs, outputs)

print("Nombre de paramètres : ", model_CNN_max.count_params())